# Random Forest Regression on the World Population

For the final test of the week, we'll learn how decision trees can be expanded upon as simple classifiers in order to create an [ensemble](https://en.wikipedia.org/wiki/Ensemble_learning) model know as a Random Forest. Like our previous coding challenges, we train this new model using the world population data from the Analyse Supplementary Exam. 

### Imports

In [0]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

In [0]:
population_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/world_population.csv', index_col='Country Code')
meta_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/metadata.csv', index_col='Country Code')

In [111]:
population_df.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,76700.0,80324.0,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100832.0,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0
AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,10604346.0,10854428.0,11126123.0,11417825.0,11721940.0,12027822.0,12321541.0,12590286.0,12840299.0,13067538.0,13237734.0,13306695.0,13248370.0,13053954.0,12749645.0,12389269.0,12047115.0,11783050.0,11601041.0,11502761.0,11540888.0,11777609.0,12249114.0,12993657.0,13981231.0,15095099.0,16172719.0,17099541.0,17822884.0,18381605.0,18863999.0,19403676.0,20093756.0,20966463.0,21979923.0,23064851.0,24118979.0,25070798.0,25893450.0,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0
AGO,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,6309770.0,6414995.0,6523791.0,6642632.0,6776381.0,6927269.0,7094834.0,7277960.0,7474338.0,7682479.0,7900997.0,8130988.0,8376147.0,8641521.0,8929900.0,9244507.0,9582156.0,9931562.0,10277321.0,10609042.0,10921037.0,11218268.0,11513968.0,11827237.0,12171441.0,12553446.0,12968345.0,13403734.0,13841301.0,14268994.0,14682284.0,15088981.0,15504318.0,15949766.0,16440924.0,16983266.0,17572649.0,18203369.0,18865716.0,19552542.0,20262399.0,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0
ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
AND,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,52448.0,54509.0,56671.0,58888.0,60971.0,62677.0,63850.0,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80991.0,82683.0,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0


In [112]:
meta_df.head()

,Region,Income Group,Special Notes
Country Code,,,
ABW,Latin America & Caribbean,High income,Mining is included in agriculture\r\r\r\nElect...
AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...
AGO,Sub-Saharan Africa,Lower middle income,NaN
ALB,Europe & Central Asia,Upper middle income,NaN
AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...


### Question 1

As we've seen previously, the world population data spans from 1960 to 2017. We'd like to build a predictive model that can give us the best guess at what the world population in a given year was. However, as a slight twist this time, we want to compute this estimate for only _countries within a given income group_. To do this, similar to our previous coding challenges, we need to partition our data such that we have testing data which is reserved for our model's evaluation.  

First, however, we need to formulate our data such that the sklearn's `RandomForestRegressor` class can train on our data. To do this, we will write a function that takes as input an income group and return a 2-d numpy array that contains the year and the measured population.

_**Function Specifications:**_
* Should take a `str` argument, called `income_group` as input and return a numpy `array` type as output.
* Set the default argument of `income_group` to equal `'Low income'`.
* If the specified value of `income_group` does not exist, the function must raise a `ValueError`.
* The array should only have two columns containing the year and the population, in other words, it should have a shape `(?, 2)` where `?` is the length of the data.
* The values within the array should be of type `np.int64`. 

_**Further Reading:**_

Data types are associated with memory allocation. As such, your choice of data type affects the precision of computations in your program. For example, the `np.int` data type in numpy can only store values between -2147483648 to 2147483647 and assigning values outside this range for variables of this data type may cause run-time errors. To avoid this, we can use data types with larger memory capacity e.g. `np.int64`.

https://docs.scipy.org/doc/numpy/user/basics.types.html

In [0]:
def get_year_pop_by_income(income_group='Low income'):
    
  x = list(['High income','Low income','Lower middle income', 'Upper middle income'])
  if income_group not in x:
    raise ValueError('income group does not exist in df')
  else:
    
    df2 = pd.DataFrame(meta_df)
    df3 = pd.DataFrame(population_df)
    df2 = df2[df2['Income Group'] == income_group]
    #df3 = df3[df3.index.sort_index() == df2.index.sort_index()]
    df3 = df3.sort_index()
    df2 = df2.sort_index()

    a = df3.index.searchsorted(df2.index)
    
    df3 = df3.iloc[a]
    df = pd.melt(df3, var_name='Year', value_name='Population')
    df =df.groupby(['Year'])[['Population']].sum()
    #df = df.drop(['Country Code'],axis=1) 
    
    array_1 = df['Population'].astype(np.int64)  
    array_2 = df.index.astype(np.int64)
    
    df=pd.DataFrame({'Year' : array_2,'Population' : array_1},dtype='int64')
    return np.array(df)
    return df3
 

In [114]:
get_year_pop_by_income('High income')

array([[      1960,  769889923],
       [      1961,  781225329],
       [      1962,  791207437],
       [      1963,  801108277],
       [      1964,  810900987],
       [      1965,  820309686],
       [      1966,  829088382],
       [      1967,  837479954],
       [      1968,  844905494],
       [      1969,  854059674],
       [      1970,  862276721],
       [      1971,  871169187],
       [      1972,  880246152],
       [      1973,  888486025],
       [      1974,  897803169],
       [      1975,  906573084],
       [      1976,  913843314],
       [      1977,  921330504],
       [      1978,  928906293],
       [      1979,  936836246],
       [      1980,  944587066],
       [      1981,  952368316],
       [      1982,  959759971],
       [      1983,  966754949],
       [      1984,  973423742],
       [      1985,  980143630],
       [      1986,  987194728],
       [      1987,  994242786],
       [      1988, 1001421456],
       [      1989, 1009036892],
       [  

_**Expected Outputs:**_
```python
get_year_pop_by_income('High income')
```
> ```
array([[      1960,  769889923],
       [      1961,  781225329],
       [      1962,  791207437],
       [      1963,  801108277],
       ...
       [      2015, 1211252041],
       [      2016, 1218629612],
       [      2017, 1225514228]])
```




### Question 2

Now that we have have our data, we need to split this into a set of variables we will be training on, and the set of variables that we will make our predictions on.

Unlike the previous coding challenges, a friend of our has indicated that sklearn has its own built-in functionality for creating training and testing sets. Here, using the `train_test_split` [method](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), we can easily shuffle and randomly choose a subset of the data as the test set.   

Using this knowledge, write a function which uses sklearn's `train_test_split` [method](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) internally, and that will take as input a 2-d numpy array and return four variables in the form of `(X_train, y_train), (X_test, y_test)`, where `(X_train, y_train)` are the features / response of the training set, and `(X-test, y_test)` are the feautes / response of the testing set. 

**Important Note:** Due to the random initialisation process used within sklearn's `train_test_split` method, you will need to fix the value of the `random_state` argument in order to get repeatable and predictable results. 


_**Function Specifications:**_
* Should take a 2-d numpy `array` as input.
* Should use sklearn's `train_test_split` [method](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
* Set `random_state` to equal `42` for this internal method.  
* Should return two `tuples` of the form `(X_train, y_train), (X_test, y_test)`.
* `(X_test, y_test)` should contain 1% of the input array. They should also be the form of an `array`, and not as a single value.


In [0]:


def sklearn_feature_response_split(arr):
  arr = np.array(arr)
  from sklearn.model_selection import train_test_split
  if arr.shape == (len(arr),2):
    X_train, X_test, y_train, y_test = train_test_split(arr[0:,0], arr[0:,1], test_size=0.01, random_state=42)
    return (np.array(X_train), np.array(y_train)), (np.array(X_test), np.array(y_test))
  else:
    return 'all'
    
 
  

In [125]:
data = get_year_pop_by_income('High income');
sklearn_feature_response_split(data)

((array([1965, 1990, 1973, 1994, 2015, 1987, 1991, 2005, 1972, 2007, 1963,
         2003, 1986, 1968, 1977, 1966, 1964, 2000, 1979, 1996, 2008, 1997,
         2013, 1975, 1969, 1976, 1984, 1993, 2014, 2012, 1985, 1971, 1992,
         2010, 2009, 1989, 2001, 1961, 1981, 1962, 2004, 1999, 1995, 1983,
         2006, 1970, 1982, 1978, 2016, 1980, 1967, 2002, 1974, 1988, 2011,
         1998]),
  array([ 820309686, 1017092667,  888486025, 1048121445, 1211252041,
          994242786, 1025345408, 1131426281,  880246152, 1149238990,
          801108277, 1115390519,  987194728,  844905494,  921330504,
          829088382,  810900987, 1092825678,  936836246, 1064630661,
         1158965286, 1071969568, 1196212921,  906573084,  854059674,
          913843314,  973423742, 1040349480, 1203819897, 1188796100,
          980143630,  871169187, 1031949811, 1175649232, 1167712409,
         1009036892, 1100293969,  781225329,  952368316,  791207437,
         1123325037, 1085992668, 1057290586,  966754949,

_**Expected Outputs:**_
```python
data = get_year_pop_by_income('High income')
sklearn_feature_response_split(data)
```
> ```
((array([1965, 1994, 1973, 2004, 2012, 1997, 1985, 2006, 1972, 2008, 1963,
         1996, 1991, 1968, 1977, 1966, 1964, 2001, 1979, 1990, 2009, 2010,
         2014, 1975, 1969, 1987, 1986, 1976, 1984, 1993, 2015, 2000, 1971,
         1992, 2016, 2003, 1989, 2013, 1961, 1981, 1962, 2005, 1999, 1995,
         1983, 2007, 1970, 1982, 1978, 2017, 1980, 1967, 2002, 1974, 1988,
         2011, 1998]),
  array([ 820309686, 1048121445,  888486025, 1123325037, 1188796100,
         1071969568,  980143630, 1140084827,  880246152, 1158965286,
          801108277, 1064630661, 1025345408,  844905494,  921330504,
          829088382,  810900987, 1100293969,  936836246, 1017092667,
         1167712409, 1175649232, 1203819897,  906573084,  854059674,
          994242786,  987194728,  913843314,  973423742, 1040349480,
         1211252041, 1092825678,  871169187, 1031949811, 1218629612,
         1115390519, 1009036892, 1196212921,  781225329,  952368316,
          791207437, 1131426281, 1085992668, 1057290586,  966754949,
         1149238990,  862276721,  959759971,  928906293, 1225514228,
          944587066,  837479954, 1107836355,  897803169, 1001421456,
         1181451343, 1078927765])),
 (array([1960]), array([769889923])))
 ```

### Question 3

Now that we have formatted our data, we can fit a model using sklearn's `DecisionTreeRegressor` class. We'll write a function that will take as input the features and response variables that we created in the last question, and return a trained model.

**Important Note:** Due to the random initialisation process used within sklearn's `DecisionTreeRegressor` class, you will need to fix the value of the `random_state` argument in order to get repeatable and predictable results.

_**Function Specifications:**_
* Should take two numpy `arrays` as input in the form `(X_train, y_train)`.
* Should return an sklearn `RandomForestRegressor` model.
* Set the `random_state` argument of the model to equal `42`
* The returned model should be fitted to the data.

_**Hint:**_
You may need to reshape the data within the function. You can use `.reshape(-1, 1)` to do this.


In [0]:
def train_model(X_train, y_train):
    from sklearn.ensemble import RandomForestRegressor
    reg = RandomForestRegressor(random_state =42, n_estimators=10)
    reg.fit(X_train.reshape(-1, 1),y_train.ravel())
    return reg

In [142]:
data = get_year_pop_by_income('High income')
(X_train, y_train), _ = sklearn_feature_response_split(data)

train_model(X_train, y_train).predict([[1960]])

array([7.86208256e+08])

_**Expected Outputs:**_
```python
train_model(X_train, y_train).predict([[1960]]) == array([7.86208256e+08])
```

### Question 4

We would now like to test on our testing data that we produced from Question 2. This test will give the Mean Absolute Error (MAE), which is given by:

$$
MAE = \frac{1}{N} \sum_{n=i}^N |p_i - y_i|
$$

where $p_i$ refers to the $i^{\rm th}$ prediction made from `X_test`, $y_i$ refers to the $i^{\rm th}$ value in `y_test`, and $N$ is the length of `y_test`.

_**Function Specifications:**_
* Should take a trained model and two `arrays` as input. This will be the `X_test` and `y_test` variables from Question 2. 
* Should return the residual sum of squares over the input from the predicted values of `X_test` as compared to values of `y_test`.
* The output should be a `float` rounded to 2 decimal places.

In [0]:
def test_model(model, X_test, y_test):
  y_pred = model.predict(X_test.reshape(-1,1))
  return np.round(np.sum(np.absolute(y_test.ravel() - y_pred)),2)

   

In [0]:
data = get_year_pop_by_income('High income')
(X_train, y_train), (X_test, y_test) = sklearn_feature_response_split(data)
lm = train_model(X_train, y_train)

In [152]:
test_model(lm, X_test, y_test)

16318333.2

_**Expected Outputs:**_
```python
test_model(lm, X_test, y_test) == 16318333.2
```